In [1]:
%load_ext autoreload
%autoreload 2


import os
os.environ['OMP_NUM_THREADS'] = '1'
import numpy as np  

# LOAD DATA

download data from s3

In [2]:
import pickle
all_data = pickle.load(open('ns_music_all_data_ming_20211124','rb'))
all_features_cpu = pickle.load(open('tracks_embedding_fullmodel_size64.p','rb'))

In [3]:
df_playlist = all_data['df_playlist']

In [4]:

val_users = all_data['val_user_ids']
train_users = all_data['train_user_ids']
df_playlist_val   = df_playlist[df_playlist['pid'].isin(val_users)]

# R-Precision

In [5]:
R =500

In [ ]:
result = []
for idx, pid in enumerate(val_users):
    
    # get all track ids of the playlist  (SORTED BY POSITION !)
    tracks = df_playlist_val[df_playlist_val['pid']==pid]
    tracks = tracks.sort_values(by='pos')
    all_tids = list(tracks['tid'])
    
    # keep the top MIN[10, len(tracks)/2 ] , predict the rest 
    known_num = min(10, len(all_tids) // 2 )   
    
    # compute average embedding to represent the playlist 
    emb = np.mean(all_features_cpu[all_tids[:known_num]], axis=0)

    # dot product to compute scores of each song, sort 
    data = np.argsort(all_features_cpu.dot(emb))[::-1]    
    
    # compute R-precision 
    predicted_best = data[:R]
    entries = all_tids[known_num:]
    portion = [x for x in entries if x in predicted_best]
    portion = len(portion)/len(entries)   
    result.append([pid, len(all_tids), known_num,  portion])
    
    # 
    if idx % 100 == 0:
        print(idx)
        if idx >  0:
            print(np.mean([x[-1] for x in result]))
            
#pickle.dump( result, open('validataion_rpres_result.p','wb'))

0
100
0.14731859306656522
200
0.137843933442304
300
0.1383373846489555
400
0.14283631011306835
500
0.14053227514146338
600
0.13828398034672054
700
0.1380339181541397
800
0.137503668348944
900
0.13619298662419344
1000
0.13806914555401592
1100
0.13832169010637893
1200
0.13702334433515398
1300
0.13669220781382718
1400
0.137630056494433
1500
0.13713543889151117
1600
0.13731003572523456
1700
0.13748828619369227
1800
0.1373068811284371
1900
0.13725338876711202
2000
0.13818151727699562
2100
0.13753865761552886
2200
0.13724762816735928
2300
0.13649173560678018
2400
0.13684146276675585
2500
0.13674422483126097
2600
0.1360814497591124
2700
0.13625925811831627
2800
0.13704751335547175
2900
0.1366413625665529
3000
0.137204756568285
3100
0.13748442098806504
3200
0.13716527687430938
3300
0.13755025637888552
3400
0.13819846922386422
3500
0.1369836645427537
3600
0.13729488976752702
3700
0.1369776995765684
3800
0.13708518158076163
3900
0.13714616708601934
4000
0.13669448553012836
4100
0.136881422608976